# TorchAudio Aligner - Test Runner

This notebook runs the pytest test suite for the torchaudio_aligner project.

**Features:**
- Clone/update repository with single click
- Install dependencies (core, full, or custom)
- Run all tests or specific test modules
- Easy re-run when code is updated

**For private repositories:**
- Option 1: Use GitHub Personal Access Token (PAT)
- Option 2: Upload repo as zip file
- Option 3: Mount Google Drive with cloned repo

## Configuration

In [ ]:
# ============================================
# Configuration - Edit these as needed
# ============================================

# Repository settings
REPO_OWNER = "huangruizhe"
REPO_NAME = "torchaudio_aligner"
BRANCH = "dev"  # or "main", "feature/xyz", etc.

# Authentication for private repos (leave empty for public repos)
# Create a token at: https://github.com/settings/tokens
# Required scope: repo (for private repos)
GITHUB_TOKEN = ""  # e.g., "ghp_xxxxxxxxxxxx"

# Installation mode: "core", "full", "dev", or "minimal"
INSTALL_MODE = "minimal"

# Test settings
TEST_VERBOSE = True      # -v flag
TEST_COVERAGE = False    # --cov flag (requires pytest-cov)
STOP_ON_FIRST_FAIL = False  # -x flag

# Specific tests to run (empty = all tests)
# Examples:
#   "tests/unit/test_aligned_word.py"
#   "tests/unit/test_aligned_word.py::TestAlignedWordTimeMethods"
#   "tests/unit/" (all unit tests)
SPECIFIC_TESTS = ""  # Leave empty to run all tests

## 1. Setup Repository

Run this cell to clone the repository (first time) or update it (subsequent runs).

In [ ]:
import os
import subprocess
from pathlib import Path

# Determine repo path
try:
    # Check if running in Colab
    import google.colab
    IN_COLAB = True
    repo_path = Path(f"/content/{REPO_NAME}")
except ImportError:
    IN_COLAB = False
    repo_path = Path(f"./{REPO_NAME}")

# Build repo URL (with or without token)
if GITHUB_TOKEN:
    REPO_URL = f"https://{GITHUB_TOKEN}@github.com/{REPO_OWNER}/{REPO_NAME}.git"
    print("Using authenticated GitHub URL (token provided)")
else:
    REPO_URL = f"https://github.com/{REPO_OWNER}/{REPO_NAME}.git"
    print("Using public GitHub URL (no token)")

def run_cmd(cmd, check=True):
    """Run shell command and return success status."""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.stdout:
        print(result.stdout)
    if result.returncode != 0:
        if result.stderr:
            # Hide token from error messages
            error = result.stderr.replace(GITHUB_TOKEN, "***") if GITHUB_TOKEN else result.stderr
            print(f"Error: {error}")
        if check:
            return False
    return True

if repo_path.exists():
    # Repository exists - update it
    print(f"Updating existing repository at {repo_path}...")
    update_cmd = f'cd {repo_path} && git fetch origin && git checkout {BRANCH} && git pull origin {BRANCH}'
    if run_cmd(update_cmd):
        print(f"\n✓ Repository updated to latest {BRANCH}")
    else:
        print(f"\n✗ Failed to update repository")
else:
    # Clone repository
    print(f"Cloning repository...")
    clone_cmd = f'git clone -b {BRANCH} {REPO_URL} {repo_path}'
    if run_cmd(clone_cmd):
        print(f"\n✓ Repository cloned to {repo_path}")
    else:
        print(f"\n✗ Failed to clone repository")
        print("\nIf this is a private repo, you need to:")
        print("1. Create a GitHub Personal Access Token at https://github.com/settings/tokens")
        print("2. Set GITHUB_TOKEN in the Configuration cell above")
        print("3. Re-run this cell")

# Show current commit (if repo exists)
if repo_path.exists():
    print("\nCurrent commit:")
    run_cmd(f'cd {repo_path} && git log -1 --oneline', check=False)

## Alternative: Upload from Google Drive or ZIP

If you can't use GitHub authentication, use one of these alternatives.

In [ ]:
# Option A: Mount Google Drive (if you have the repo there)
# Uncomment and run if needed:

# from google.colab import drive
# drive.mount('/content/drive')
# repo_path = Path('/content/drive/MyDrive/torchaudio_aligner')
# print(f"Using repo from Google Drive: {repo_path}")

In [ ]:
# Option B: Upload ZIP file
# Uncomment and run if needed:

# from google.colab import files
# import zipfile
# 
# print("Upload your repository as a ZIP file...")
# uploaded = files.upload()
# 
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('/content/')
#         print(f"Extracted {filename}")
#         # Update repo_path to match extracted folder name
#         repo_path = Path('/content/torchaudio_aligner')  # adjust if needed

## 2. Install Dependencies

In [ ]:
import sys

if not repo_path.exists():
    print(f"Error: Repository not found at {repo_path}")
    print("Please run the Setup Repository cell first.")
else:
    # Install based on mode
    if INSTALL_MODE == "minimal":
        # Just pytest, assume torch is already available (Colab)
        !pip install -q pytest pytest-cov
        print("✓ Minimal dependencies installed (pytest only)")
        
    elif INSTALL_MODE == "core":
        # Core dependencies for most functionality
        !pip install -q -e "{repo_path}[core,test]"
        print("✓ Core dependencies installed")
        
    elif INSTALL_MODE == "full":
        # Full installation
        !pip install -q -e "{repo_path}[full,test]"
        print("✓ Full dependencies installed")
        
    elif INSTALL_MODE == "dev":
        # Development dependencies
        !pip install -q -e "{repo_path}[full,dev]"
        print("✓ Development dependencies installed")
        
    else:
        print(f"Unknown install mode: {INSTALL_MODE}")
        print("Using requirements.txt as fallback...")
        !pip install -q -r {repo_path}/requirements.txt
        !pip install -q pytest pytest-cov

    # Add src to path
    src_path = str(repo_path / "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✓ Added {src_path} to Python path")

## 3. Verify Installation

In [ ]:
# Quick verification that imports work
print("Verifying imports...")

try:
    import torch
    print(f"  ✓ torch {torch.__version__}")
except ImportError as e:
    print(f"  ✗ torch: {e}")

try:
    import torchaudio
    print(f"  ✓ torchaudio {torchaudio.__version__}")
except ImportError as e:
    print(f"  ✗ torchaudio: {e}")

try:
    from alignment.base import AlignedWord, AlignmentResult
    print(f"  ✓ alignment.base")
except ImportError as e:
    print(f"  ✗ alignment.base: {e}")

try:
    from api import align_long_audio
    print(f"  ✓ api")
except ImportError as e:
    print(f"  ✗ api: {e}")

print("\nReady to run tests!")

## 4. Run Tests

In [ ]:
# Build pytest command
pytest_args = []

if TEST_VERBOSE:
    pytest_args.append("-v")

if TEST_COVERAGE:
    pytest_args.append(f"--cov={repo_path}/src")
    pytest_args.append("--cov-report=term-missing")

if STOP_ON_FIRST_FAIL:
    pytest_args.append("-x")

# Test path
if SPECIFIC_TESTS:
    test_path = f"{repo_path}/{SPECIFIC_TESTS}"
else:
    test_path = f"{repo_path}/tests/"

pytest_args.append(test_path)

# Build command
args_str = " ".join(pytest_args)
cmd = f"PYTHONPATH={repo_path}/src python -m pytest {args_str}"

print(f"Running: {cmd}\n")
print("=" * 60)
!{cmd}

## 5. Run Specific Test Categories

Use these cells to run specific test categories.

In [ ]:
# Run only unit tests
!PYTHONPATH={repo_path}/src python -m pytest {repo_path}/tests/unit/ -v

In [ ]:
# Run only integration tests
!PYTHONPATH={repo_path}/src python -m pytest {repo_path}/tests/integration/ -v

In [ ]:
# Run alignment.base tests only
!PYTHONPATH={repo_path}/src python -m pytest {repo_path}/tests/unit/test_aligned_word.py {repo_path}/tests/unit/test_aligned_char.py {repo_path}/tests/unit/test_alignment_result.py -v

In [ ]:
# Run text normalization tests
!PYTHONPATH={repo_path}/src python -m pytest {repo_path}/tests/unit/test_text_normalization.py -v

In [ ]:
# List all available tests without running them
!PYTHONPATH={repo_path}/src python -m pytest {repo_path}/tests/ --collect-only -q

## 6. Quick Update & Re-run

Use this cell to quickly pull updates and re-run all tests.

In [ ]:
# Quick update and test
print("Pulling latest changes...")
!cd {repo_path} && git fetch origin && git checkout {BRANCH} && git pull origin {BRANCH}
print("\nCurrent commit:")
!cd {repo_path} && git log -1 --oneline
print("\n" + "=" * 60)
print("Running tests...\n")
!PYTHONPATH={repo_path}/src python -m pytest {repo_path}/tests/ -v

## 7. Manual Testing

Use this cell for quick manual tests.

In [ ]:
# Quick manual test
from alignment.base import AlignedWord, AlignmentResult

# Test AlignedWord
word = AlignedWord(word="hello", start_frame=100, end_frame=150)
print(f"Word: {word.word}")
print(f"Start: {word.start_seconds()}s (frame {word.start_frame})")
print(f"End: {word.end_seconds()}s (frame {word.end_frame})")
print(f"Duration: {word.duration_seconds()}s ({word.duration_frames} frames)")
print(f"\nRepr: {repr(word)}")

# Test AlignmentResult
words = [
    AlignedWord(word="hello", start_frame=100, end_frame=150, index=0),
    AlignedWord(word="world", start_frame=160, end_frame=220, index=1),
]
result = AlignmentResult(words=words)
print(f"\nAlignmentResult:")
print(f"  Text: {result.text}")
print(f"  Num words: {len(result)}")
print(f"  Duration: {result.duration}s")